## Criando um data frame e utilizando WithColumn 2 - Aula 5
1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic
2. Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor com a seguinte estrutura:

ANUAL | MESES | VALOR MÉDIO | VALOR MÍNIMO | VALOR MÁXIMO
2019  | 2     | 00.00       | 00.00        | 00.00
2018  | 12    | 00.00       | 00.00        | 00.00
2017  | 12    | 00.00       | 00.00        | 00.00
...   | ...   | 00.00       | 00.00        | 00.00
1986  | 2     | 00.00       | 00.00        | 00.00

3. Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc

In [1]:
#verificando se o caminho do arquivo está correto
!hdfs dfs -ls /user/ronnan/data/juros_selic/juros_selic/

-rw-r--r--   3 root supergroup       7954 2022-04-06 03:19 /user/ronnan/data/juros_selic/juros_selic


In [2]:
#lendo o arquivo juros_selic
!hdfs dfs -cat /user/ronnan/data/juros_selic/juros_selic/

"data";"valor"
"01/06/1986";"1,27"
"01/07/1986";"1,95"
"01/08/1986";"2,57"
"01/09/1986";"2,94"
"01/10/1986";"1,96"
"01/11/1986";"2,37"
"01/12/1986";"5,47"
"01/01/1987";"11,00"
"01/02/1987";"19,61"
"01/03/1987";"11,95"
"01/04/1987";"15,30"
"01/05/1987";"24,63"
"01/06/1987";"18,02"
"01/07/1987";"8,91"
"01/08/1987";"8,09"
"01/09/1987";"7,99"
"01/10/1987";"9,45"
"01/11/1987";"12,92"
"01/12/1987";"14,38"
"01/01/1988";"16,78"
"01/02/1988";"18,35"
"01/03/1988";"16,59"
"01/04/1988";"20,25"
"01/05/1988";"18,65"
"01/06/1988";"20,17"
"01/07/1988";"24,69"
"01/08/1988";"22,63"
"01/09/1988";"26,25"
"01/10/1988";"29,79"
"01/11/1988";"28,41"
"01/12/1988";"30,24"
"01/01/1989";"22,97"
"01/02/1989";"18,95"
"01/03/1989";"20,41"
"01/04/1989";"11,52"
"01/05/1989";"11,43"
"01/06/1989";"27,29"
"01/07/1989";"33,15"
"01/08/1989";"35,49"
"01/09/1989";"38,58"
"01/10/1989";"47,70"
"01/11/1989";"48,41"
"01/12/1989";"64,21"
"01/01/1990";"67,60"
"01/02/1990";"82,04"
"01/03/1990";"36,76"
"01/04/1990";"4,23"
"01/05/199

In [5]:
#1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic
juros = spark.read.csv("/user/ronnan/data/juros_selic/juros_selic",sep= ";",header=True)



In [7]:
#visualiar o schema e visualizar os 5 primeiros
print(juros.printSchema())
juros.show(5)


root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)

None
+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [18]:
from pyspark.sql.functions import *
from pyspark.sql.tests import *

In [16]:
#2. Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor com a seguinte estrutura:
juros_ano = juros.withColumn("ano", split(col("data"),"/").getItem(2))


In [19]:
#substituindo a virgula por ponto do campo valor / use  cast para mudar o formato de string para Float
# lembre se de importar o types
juros_valor = juros_ano.withColumn("valor",regexp_replace(col("valor"),"\,","\.").cast(FloatType()))
juros_valor.printSchema() #note que o campo valor foi anterado de string para Float(valor com virgula)

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)



In [42]:
# utilizando o agg para somar com count os meses, depois alias para renomear coluna
# avg para inserir a media e alias para renomear coluna e format_number para reduzir 2 casas
# min para acrescentar o valor minimo
# max para acrescentar o valor maximo
#sort para ordenar pelo ano desc(descrescente)

juros_relatorio = juros_valor.groupBy("ano").agg(count("ano").alias("Meses"),
format_number(avg("valor"),2).alias("Valor Médio"), min("valor").alias("valor Mínimo"),
max("valor").alias("Valor Máximo")).sort(desc("ano"))


In [44]:
#3. Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc
juros_relatorio.write.orc("/user/ronnan/relatorio_anual", compression= "zlib")

In [89]:
!hdfs dfs -ls /user/ronnan/relatorio_anual

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-10 21:21 /user/ronnan/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        585 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00000-bdaefbb4-c1de-4612-ae17-8132b62564be-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00001-bdaefbb4-c1de-4612-ae17-8132b62564be-c000.zlib.orc
-rw-r--r--   2 root supergroup        588 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00002-bdaefbb4-c1de-4612-ae17-8132b62564be-c000.zlib.orc
-rw-r--r--   2 root supergroup        575 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00003-bdaefbb4-c1de-4612-ae17-8132b62564be-c000.zlib.orc
-rw-r--r--   2 root supergroup        586 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00004-bdaefbb4-c1de-4612-ae17-8132b62564be-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2022-04-10 21:21 /user/ronnan/relatorio_anual/part-00005-bdaefbb4-c1de-4612-ae17-8132b62564be-c0

In [88]:
#
!hdfs dfs -ls /user/ronnan/relatorio_anual_particionado

Found 35 items
-rw-r--r--   2 root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/_SUCCESS
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1986
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1987
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1988
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1989
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1990
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1991
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=1992
drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=

In [80]:






!hdfs dfs -ls /user/hive/warehouse/








Found 1 items
drwxr-xr-x   - root supergroup          0 2022-04-08 00:39 /user/hive/warehouse/juros


In [84]:








#3 alternativa("clayton"). Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc
# para criar partições com base no campo ano partitionBy
juros_relatorio.write.orc("/user/ronnan/relatorio_anual_particionado", partitionBy="ano", compression= "zlib")










In [85]:
# verificar se o arquivo no ano de 2018 pode ser localizar
!hdfs dfs -ls /user/ronnan/relatorio_anual_particionado | grep 2018

drwxr-xr-x   - root supergroup          0 2022-04-10 21:44 /user/ronnan/relatorio_anual_particionado/ano=2018


In [86]:
# salvando a tabela relatorio dentro do hive conforme as partições
juros_hive = juros_relatorio.withColumnRenamed("Valor médio", "valor_medio").withColumnRenamed("Valor Máximo","valor_maximo").withColumnRenamed("Valor Mínimo","valor_minimo")
juros_hive.write.saveAsTable("relatorioanualparticionado", partitionBy="ano", mode="OverWrite")












Py4JJavaError: An error occurred while calling o957.saveAsTable.
: java.lang.NoClassDefFoundError: parquet/hadoop/ParquetOutputFormat
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:238)
	at org.apache.spark.sql.hive.client.HiveClientImpl$.org$apache$spark$sql$hive$client$HiveClientImpl$$toOutputFormat(HiveClientImpl.scala:913)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$toHiveTable$6.apply(HiveClientImpl.scala:947)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$toHiveTable$6.apply(HiveClientImpl.scala:947)
	at scala.Option.map(Option.scala:146)
	at org.apache.spark.sql.hive.client.HiveClientImpl$.toHiveTable(HiveClientImpl.scala:947)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply$mcV$sp(HiveClientImpl.scala:482)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply(HiveClientImpl.scala:480)
	at org

In [87]:
spark.read.table("relatorioanualparticionado")










AnalysisException: 'Table or view not found: relatorioanualparticionado;'